### Load Library ###

In [1]:
import sys
import os
import numpy as np
import torch
import matplotlib.pyplot as plt

### Set Up Dataset ###
Download nuScenes dataset and uncompress it.

In [2]:
# Initialization
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from nuimages import NuImages

# Load NuImages
nuim_train = NuImages(dataroot='/root/data/nuimages/images/', version='v1.0-train', verbose=True, lazy=False)
nuim_val = NuImages(dataroot='/root/data/nuimages/images/', version='v1.0-val', verbose=True, lazy=False)

Loading nuImages tables for version v1.0-train...
Loaded 12 attribute(s) in 0.015s,
Loaded 2022 calibrated_sensor(s) in 0.073s,
Loaded 25 category(s) in 0.000s,
Loaded 872181 ego_pose(s) in 7.313s,
Loaded 350 log(s) in 0.018s,
Loaded 557715 object_ann(s) in 5.346s,
Loaded 67279 sample(s) in 0.157s,
Loaded 872181 sample_data(s) in 4.993s,
Loaded 6 sensor(s) in 0.001s,
Loaded 73755 surface_ann(s) in 3.166s,
Done loading in 21.081 seconds (lazy=False).
Loading nuImages tables for version v1.0-val...
Loaded 12 attribute(s) in 0.000s,
Loaded 464 calibrated_sensor(s) in 0.014s,
Loaded 25 category(s) in 0.000s,
Loaded 213185 ego_pose(s) in 1.392s,
Loaded 82 log(s) in 0.018s,
Loaded 136074 object_ann(s) in 1.722s,
Loaded 16445 sample(s) in 0.057s,
Loaded 213185 sample_data(s) in 1.221s,
Loaded 6 sensor(s) in 0.019s,
Loaded 17995 surface_ann(s) in 0.823s,
Done loading in 5.268 seconds (lazy=False).


In [7]:
dataroot = '/root/data/nuimages/images/'
attribute_dict = {}
for attri in nuim_val.attribute:
    attribute_dict[attri['token']] = attri['name']

# Attributes for samples in training set
data_attributes = {}
for ob in nuim_val.object_ann:
    sample_token = ob['sample_data_token']

    # Get object's file name
    path = dataroot + nuim_val.get('sample_data', sample_token)['filename']
    if path not in data_attributes and ob['attribute_tokens']:
        data_attributes[path] = set()
    
    # Get object's attribute
    for attribute_token in ob['attribute_tokens']:
        data_attributes[path].add(attribute_dict[attribute_token])

Create a .txt file discripting where the files are stored.

In [5]:


path_with = '/root/siqi/Feature-Mining-for-Object-Detection/split_result/with_rider_val.txt'
path_without = '/root/siqi/Feature-Mining-for-Object-Detection/split_result/without_rider_val.txt'
path_other = '/root/siqi/Feature-Mining-for-Object-Detection/split_result/other_rider_val.txt'

target_1 = 'cycle.with_rider'
target_2 = 'cycle.without_rider'


f1 = open(path_with, 'w')
f2 = open(path_without, 'w')
f3 = open(path_other, 'w')

for file in data_attributes.keys():
    if target_1 in data_attributes[file] and target_2 not in data_attributes[file]:
        f1.write(file + '\n')

    elif target_1 not in data_attributes[file] and target_2 in data_attributes[file]:
        f2.write(file + '\n')
    
    elif target_1 not in data_attributes[file] and target_2 not in data_attributes[file]:
        f3.write(file + '\n')

Check weird data

In [6]:
category_dict = {}
for cate in nuim_val.category:
    category_dict[cate['token']] = cate['name']

cycle_with_rider = '271f6773e4d2496cbb9942c204c8a4c1'
cycle_without_rider = '725f0121878f4c86909f256fd9df8569'

for ob in nuim_val.object_ann:
    print(ob)
    for attribute_token in ob['attribute_tokens']:
        if 

{'token': '0000efb0e1c841d49ddfb26734c38fb1', 'category_token': '6021b5187b924d64be64a702e5570edf', 'bbox': [292, 454, 405, 508], 'mask': {'size': [900, 1600], 'counts': 'ZVNRODhkazA+QzdIOEw0TjIwMDAwMDAwMDA3STJOMDAwMDAwMU9PMk8xTzBPN0oxTzFOMTAxTjJPME8yTjJPME8xMDAwMDAxTzBPMTAwTzFWT1JVTz5cazBNM04xTjEwTzAwMU8xTzAxME8wMDEwTzAxTjFOM05IVV47TFdiRDJOMk4yTzEwMDAwMDAwMDAwMDAwMDAwMDFPTjJOMk5YWmNRMQ=='}, 'attribute_tokens': ['abc0f113547848a9baaa62096fca37f5'], 'sample_data_token': '88203d2859ab4dac9870d5559a99ce9e'}
{'63a94dfa99bb47529567cd90d3b58384': 'animal', 'a86329ee68a0411fb426dcad3b21452f': 'flat.driveable_surface', '1fa93b757fc74fb197cdd60001ad8abf': 'human.pedestrian.adult', 'b1c6de4c57f14a5383d9f963fbdcb5cb': 'human.pedestrian.child', '909f1237d34a49d6bdd27c2fe4581d79': 'human.pedestrian.construction_worker', '403fede16c88426885dd73366f16c34a': 'human.pedestrian.personal_mobility', 'e3c7da112cd9475a9a10d45015424815': 'human.pedestrian.police_officer', '6a5888777ca14867a8aee3fe539b56c4': 

In [8]:
print(attribute_dict)

{'271f6773e4d2496cbb9942c204c8a4c1': 'cycle.with_rider', '725f0121878f4c86909f256fd9df8569': 'cycle.without_rider', '963614d0532a4c6e946d303a94f40a3e': 'pedestrian.moving', '8c92f43bdb7c4df399aac34068f08f0f': 'pedestrian.sitting_lying_down', '60346f6d0fcd4d9eadd6f64c77dd1e93': 'pedestrian.standing', 'abc0f113547848a9baaa62096fca37f5': 'vehicle.moving', '9f65c1eaa74e4d5db46e87a34811e994': 'vehicle.parked', '46dfc76161234ff1a74eff81da7daab0': 'vehicle.stopped', '3ba0a4fefa8a46f39118e35e45c4111a': 'vehicle_light.emergency.flashing', '67125c542a1e4e25aa05dcf1c3a282b9': 'vehicle_light.emergency.not_flashing', 'f8cc93a317e04f098423bedb0150e35a': 'vertical_position.off_ground', 'ca7cd5668bac48c6b670c43fc4c6f21b': 'vertical_position.on_ground'}
